In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


In [8]:
temp_data_train = pd.read_csv('../Data/train_data.csv')
temp_data_test = pd.read_csv('../Data/test_data.csv')
# Convert the 'date' column to datetime
temp_data_train['date'] = pd.to_datetime(temp_data_train['date'])
temp_data_test['date'] = pd.to_datetime(temp_data_test['date'])

T_actual_train = temp_data_train['mean'].to_numpy()
T_actual_test = temp_data_test['mean'].to_numpy()

temp_data_test.head()

,date,mean,max,min
0,2024-04-01,10.33,20.39,3.61
1,2024-04-02,13.22,24.22,4.50
2,2024-04-03,14.72,24.39,3.72
3,2024-04-04,12.28,24.22,5.11
4,2024-04-05,8.22,16.89,3.50


Linear and Seasonal forecast

In [ ]:
# 1. Forecast settings
forecast_end = 365  # number of forecast time steps
forecast_start = 0

# 2. Create a dataframe empty for the forecast
forecast_df = pd.DataFrame(columns=['t', 'trend', 'seasonal', 'forecast', 'actual'])
forecast_df['actual'] = T_actual_test [:forecast_end]

forecast_df['t'].iloc[0] = temp_data_train['date'].iloc[-1]
date_range = pd.date_range(start=temp_data_train['date'].iloc[-1], periods=forecast_end+1)
forecast_df['t'] = date_range[1:]

print(forecast_df.head())

intercept , slope = 0, 0

# Forecast trend using linear extrapolation:
forecast_df['trend'] = intercept + slope * t

forecast_df.head()  

In [ ]:
def forecast_seasonal(t_future, fitted_params):
    """
    Forecast seasonal component using fitted sinusoidal model
    
    Parameters:
    t_future: array-like, future time points to forecast
    fitted_params: list or array containing [a1, phi1, a2, phi2, a3, phi3, a4, phi4]
    
    Returns:
    array of forecasted seasonal values
    """
    a1, phi1, a2, phi2, a3, phi3, a4, phi4 = fitted_params
    p1, p2, p3 = 365, 730, 1460
    p4 = 2920  # ~8 years modulation period
    
    # Calculate each component
    cycle1 = a1 * np.sin(2 * np.pi * (t_future - phi1) / p1)
    cycle2 = a2 * np.sin(2 * np.pi * (t_future - phi2) / p2)
    cycle3 = a3 * np.sin(2 * np.pi * (t_future - phi3) / p3)
    modulated = a4 * (1 + np.sin(2 * np.pi * (t_future - phi4) / p4)) * np.sin(2 * np.pi * t_future / 365)
    
    return cycle1 + cycle2 + cycle3 + modulated

# Example usage:
# Create future time points (e.g., next 365 days)
last_t = forecast_df['t'].max()
t_future = np.arange(last_t + 1, last_t + 366)

# Your fitted parameters from the model
fitted_params = [6.222, 44.85,    # a1, phi1
                0.640, 206.55,    # a2, phi2
                0.977, 2119.84,   # a3, phi3
                -0.2277, 423.1776]         # Add your fitted a4, phi4 values

# Generate forecast
seasonal_forecast = forecast_seasonal(t_future, fitted_params)
# Add seasonal forecast to the forecast dataframe
forecast_df['seasonal'] = seasonal_forecast[:forecast_end]